In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import mlflow

# Read recipe inputs
model = dataiku.Folder("custom_rf")

import joblib
import io
#active_model = dataiku.get_custom_variables()["active_model"]
active_model = "clf.pkl"
EVALUATION_DATASET = "clv_train_test"

with model.get_download_stream(active_model) as stream:
    bs = io.BytesIO(stream.read())
    clf = joblib.load(bs)

# Write recipe outputs
path = dataiku.Folder("mlflow_model").get_path()
model_dir = os.path.join(path, active_model)
mlflow.sklearn.save_model(clf, model_dir)

# Import MLFlow model
client = dataiku.api_client()
project = client.get_default_project()
saved_model = project.create_mlflow_pyfunc_model("model", "BINARY_CLASSIFICATION")
mlflow_version = saved_model.import_mlflow_version_from_managed_folder(
    "v1",
    "ojs6BL42",
    active_model,
    "py_38_mlflow")
mlflow_version.set_core_metadata("target", class_labels=['0', '1'], get_features_from_dataset=EVALUATION_DATASET)
mlflow_version.evaluate(EVALUATION_DATASET)